In [ ]:
# 单元格 1: 加载模型和分词器

import torch
import numpy as np
from transformers import AutoTokenizer

# 假设 config.py 和 model.py 在同一个目录下或在 Python 路径中
import config
from model import Qwen2ForSC2Fusion

def load_model_and_tokenizer():
    """
    加载并初始化 SC2Fusion 模型和分词器。
    返回:
        tuple: (model, tokenizer)
    """
    # --- 1. 设置模型路径 ---
    # 请确保这个路径是正确的
    save_path = '/data4/SC2/SC2_units_token_compress/model/Qwen_1_5B_MLP_modify_4_epochs'
    print(f"正在从 '{save_path}' 加载已训练的模型...")

    # --- 2. 加载分词器 ---
    # 使用 use_fast=False 避免 tokenizer.json 的潜在问题
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            save_path,
            trust_remote_code=True,
            local_files_only=True,
            use_fast=False  # 绕过 fast tokenizer 的加载问题
        )
    except Exception as e:
        print(f"加载分词器时出错: {e}")
        raise

    # --- 3. 加载自定义模型 ---
    model = Qwen2ForSC2Fusion.from_pretrained(
        save_path,
        entity_vector_dim=config.ENTITY_VECTOR_DIM, # 提供自定义参数
        trust_remote_code=True,
        local_files_only=True
    )

    # --- 4. 初始化模型 ---
    sc2_token_id = tokenizer.convert_tokens_to_ids(config.SC2_ENTITY_TOKEN)
    model.sc2_entity_token_id = sc2_token_id

    device = torch.device(config.DEVICE)
    model.to(device)
    model.eval()
    
    print("模型和分词器加载成功！")
    return model, tokenizer

# --- 执行加载 ---
# model 和 tokenizer 变量将在此单元格的全局作用域中可用
model, tokenizer = load_model_and_tokenizer()

In [5]:
data =   {
    "encode": [
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      1,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "text_dict": {
      "if_ours": true,
      "name": "MissileTurret",
      "health": 0,
      "Position": {
        "x": 0,
        "y": 40
      }
    }
  }

In [16]:
# 单元格 2: 定义推理函数并生成内容

def generate_content(model, tokenizer, prompt: str, entity_vector: list) -> str:
    """
    使用已加载的模型和分词器生成内容。
    """
    device = model.device

    # --- 1. 准备输入 ---
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    entity_vector_tensor = torch.tensor(entity_vector, dtype=torch.float32).unsqueeze(0).to(device)

    # --- 2. 生成回复 ---
    print("正在生成回复...")
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            entity_vectors=entity_vector_tensor,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    # --- 3. 解码并返回结果 ---
    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response

# --- 准备推理数据 ---
# test_prompt = (
#     """<sc2_entity>\nThis is an entity in a StarCraft scene. The entity's attributes include:\n1.  Whether it is a friendly entity\n2.  The entity's name\n3.  The entity's health points\n4.  The entity's position\n\nPlease output the entity's attributes in a structured way.\nExample: [True, # Friendly entity, if not a friendly entity then it is False\n\"xxxxx\", # The entity's name\n100, # The entity's health points\n{\"x\": 10, \"y\": 20}] # The entity's position\n"""
# )

test_prompt = (
    "<sc2_entity>\n 以上是一个 星际争霸场景的entity，请你描述一下 这个entity的状态"
)

print(f"实体向量维度: {config.ENTITY_VECTOR_DIM}")
# test_entity_vector = np.random.rand(config.ENTITY_VECTOR_DIM).tolist()
test_entity_vector = data["encode"]

# --- 执行推理 ---
response = generate_content(model, tokenizer, test_prompt, test_entity_vector)

# --- 打印结果 ---
print("\n" + "="*50)
print("输入 Prompt:")
print(test_prompt)
print("\n" + "="*50)
print("模型生成结果:")
print(response)
print("="*50)
print(data["text_dict"])

实体向量维度: 83
正在生成回复...

输入 Prompt:
<sc2_entity>
 以上是一个 星际争霸场景的entity，请你描述一下 这个entity的状态

模型生成结果:
或者属性。 航空母

一艘 航空母·

在 星际争霸场景·

中处于 昏暗的光线·

下。// 航空母·

可以发射 航空母炮·

。// 航空母炮·

可以攻击 目标·

。// 目标·

是一个 战舰·

。// 航空母炮·

可以攻击 艇队·

。// 艇队·

是一个 班队·

。// 班队·

是一个 艇队·

。// 艇队·

可以攻击 危险区域·

。// 危险区域·

是一个 地图区域·

。// 地图区域·

是一个 地图·

。// 地图·

是一个 地图。// 地图·

可以进行 航空母炮·

攻击。// 地图区域·

是 航空母炮·

攻击的目标。// 地图·

是 地图区域·

的容器。// 地图区域·

是 危险区域·


{'if_ours': False, 'name': 'Marine', 'health': 80, 'Position': {'x': 20, 'y': 30}}
